## Esse notebook apresenta a previsão dos próximos hits do spotify

Importação das tabelas

In [ ]:
import pandas as pd

# Carregar os dados de treino e teste
train_path = 'train.csv'
test_path = 'test.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Ver as primeiras linhas dos datasets
train_data_head = train_data.head()
test_data_head = test_data.head()

# Verificar informações gerais do dataset de treino
train_info = train_data.info()

train_data_head, test_data_head, train_info


# Limpeza de dados

In [ ]:
# Verificar a existência de valores nulos nos datasets
missing_train = train_data.isnull().sum()
missing_test = test_data.isnull().sum()

# Análise estatística dos dados numéricos
train_description = train_data.describe()

missing_train, missing_test, train_description

# Codificação de variáveis categóricas

In [ ]:
# Tratamento de valores nulos: preencher com string vazia para colunas categóricas
test_data['artists'].fillna('', inplace=True)
test_data['album_name'].fillna('', inplace=True)
test_data['track_name'].fillna('', inplace=True)

# Codificação de variáveis categóricas: vamos começar com 'explicit' e 'mode' (que são binárias)
train_data['explicit'] = train_data['explicit'].astype('category')
train_data['mode'] = train_data['mode'].astype('category')

test_data['explicit'] = test_data['explicit'].astype('category')
test_data['mode'] = test_data['mode'].astype('category')

# Converter 'explicit' e 'mode' para numéricas
train_data['explicit'] = train_data['explicit'].cat.codes
train_data['mode'] = train_data['mode'].cat.codes

test_data['explicit'] = test_data['explicit'].cat.codes
test_data['mode'] = test_data['mode'].cat.codes

# Verificar a codificação
train_data[['explicit', 'mode']].head(), test_data[['explicit', 'mode']].head()


# Normalização dos dados

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Definir colunas numéricas para escalonamento
numeric_cols = ['duration_ms', 'danceability', 'energy', 'loudness', 'speechiness', 
                'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'acousticness']

# Aplicar MinMaxScaler para normalizar os dados numéricos entre 0 e 1
scaler = MinMaxScaler()

train_data[numeric_cols] = scaler.fit_transform(train_data[numeric_cols])
test_data[numeric_cols] = scaler.transform(test_data[numeric_cols])

# Codificação de 'track_genre' usando o método 'category'
train_data['track_genre'] = train_data['track_genre'].astype('category').cat.codes
test_data['track_genre'] = test_data['track_genre'].astype('category').cat.codes

# Verificar as mudanças nas colunas numéricas e de gênero
train_data[numeric_cols + ['track_genre']].head(), test_data[numeric_cols + ['track_genre']].head()


# Exploração de dados

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plotar a distribuição da variável alvo
sns.countplot(x='popularity_target', data=train_data)
plt.title('Distribuição da Popularidade')
plt.show()


O gráfico acima mostra que a diferença entre as músicas populares e não populares é pequena, o que indica uma distribuição quase que igual entre essas targets

Correlação dos dados

In [ ]:
# Calcular a correlação
corr_matrix = train_data[numeric_cols + ['popularity_target']].corr()

# Exibir um heatmap das correlações
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlação entre Características Acústicas e Popularidade')
plt.show()


# Hipoteses

Hipótese 1: Músicas com alta energia têm mais chances de serem populares

Justificativa: Músicas com maior energia tendem a ser mais animadas, o que pode atrair mais ouvintes, especialmente em playlists populares.

In [ ]:
# Definir um limiar para alta e baixa energia
energia_limite = 0.5  # Definimos 0.5 como o ponto de corte

# Contar o número de músicas populares em cada nível de energia
energy_popularity_counts = train_data.groupby(
    [train_data['energy'].apply(lambda x: 'Alta Energia' if x >= energia_limite else 'Baixa Energia'),
     'popularity_target']
).size().unstack()

# Plotar o gráfico de barras
energy_popularity_counts.plot(kind='bar', stacked=True, figsize=(10, 6), color=['#1f77b4', '#ff7f0e'])
plt.title('Quantidade de Músicas Populares por Nível de Energia')
plt.xlabel('Nível de Energia')
plt.ylabel('Quantidade de Músicas')
plt.legend(['Não Popular', 'Popular'], title='Popularidade')
plt.xticks(rotation=0)
plt.show()


Esse gráfico mostra que dentre as músicas populares, a maioria delas são de baixa energia. Sendo assim, a hipótese é rejeitada.

Hipótese 2: Músicas mais curtas têm maior chance de se tornarem virais

Justificativa: Músicas com menor duração podem manter o interesse dos ouvintes em plataformas de streaming, onde a atenção é disputada.

In [ ]:
# Definir um limiar para músicas curtas e longas (3 minutos = 180000 ms)
duracao_limite = 180000  # 3 minutos

# Contar o número de músicas populares em cada nível de duração
duration_popularity_counts = train_data.groupby(
    [train_data['duration_ms'].apply(lambda x: 'Curta' if x <= duracao_limite else 'Longa'), 
     'popularity_target']
).size().unstack()

# Plotar o gráfico de barras
duration_popularity_counts.plot(kind='bar', stacked=True, figsize=(10, 6), color=['#1f77b4', '#ff7f0e'])
plt.title('Quantidade de Músicas Populares por Duração (Curta vs Longa)')
plt.xlabel('Duração da Música')
plt.ylabel('Quantidade de Músicas')
plt.legend(['Não Popular', 'Popular'], title='Popularidade')
plt.xticks(rotation=0)
plt.show()


O gráfico mostra que dentre as músicas curtas, há mais músicas não populares do que populares. Sendo assim, a hipótese é rejeitada.

Hipótese 3: Músicas mais dançantes (danceability) têm maior chance de se tornarem populares.

A hipótese aqui é que músicas com características que as tornam boas para dançar (alta danceability) são mais atraentes para o público e, portanto, têm uma maior chance de viralizar.

In [ ]:
# Criar contagem de músicas populares por nível de dançabilidade
danceability_popularity_counts = train_data.groupby(
    [train_data['danceability'].apply(lambda x: 'Alta Danceability' if x >= 0.5 else 'Baixa Danceability'), 
     'popularity_target']
).size().unstack()

# Plotar o gráfico de barras empilhadas
danceability_popularity_counts.plot(kind='bar', stacked=True, figsize=(10, 6), color=['#1f77b4', '#ff7f0e'])
plt.title('Quantidade de Músicas Populares por Nível de Dançabilidade')
plt.xlabel('Nível de Dançabilidade')
plt.ylabel('Quantidade de Músicas')
plt.legend(['Não Popular', 'Popular'], title='Popularidade')
plt.xticks(rotation=0)
plt.show()


É possível ver que a maioria das músicas populares são de baixa dançabilidade, mas ao mesmo tempo, a a maioria das não populares também são. Isso indica que há um equilibrio entre músicas de alta ou baixa dançabilidade em relação a popularidade

# Seleção de features

In [10]:
from sklearn.model_selection import train_test_split

# Separar os dados entre features e target
X = train_data.drop(columns=['popularity_target', 'track_unique_id', 'track_id', 'artists', 'album_name', 'track_name'])
y = train_data['popularity_target']

# Dividir entre treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Construção e Avaliação do Modelo 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Instanciar o modelo
rf_model = RandomForestClassifier(random_state=42)

# Treinar o modelo
rf_model.fit(X_train, y_train)

# Fazer previsões
y_pred = rf_model.predict(X_val)

# Avaliar o modelo
accuracy = accuracy_score(y_val, y_pred)
print(f'Acurácia: {accuracy:.2f}')


In [12]:
# Fazer previsões no conjunto de teste
X_test = test_data.drop(columns=['track_unique_id', 'track_id', 'artists', 'album_name', 'track_name'])
test_predictions = rf_model.predict(X_test)

# Gerar arquivo CSV para submissão
submission = test_data[['track_unique_id']].copy()
submission['popularity_target'] = test_predictions
submission.to_csv('submission.csv', index=False)


# Finetuning de Hiperparâmetros

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None]
}

grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(f'Melhores parâmetros: {grid_search.best_params_}')


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Fazer previsões no conjunto de validação usando o melhor modelo encontrado
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)

# Calcular a acurácia
accuracy = accuracy_score(y_val, y_pred)
print(f'Acurácia com o modelo otimizado: {accuracy:.2f}')

# Exibir outras métricas de avaliação
print("Relatório de Classificação:\n", classification_report(y_val, y_pred))
